In [2]:
import numpy as np
import cv2

In [3]:
#corner_track_params = dict(maxCorners=10, qualityLevel=0.3, minDistance=7, blockSize=7) #start with that and adjuste for better use
corner_track_params = dict(maxCorners=100, qualityLevel=0.01, minDistance=10, blockSize=7)

In [4]:
#small windows loss big movements, big windows loss smal movements, so adjuste for better use
#maxLevel, https://en.wikipedia.org/wiki/Pyramid_(image_processing)
#the values bellow can exchange speed X accuracy
#two criterias to perform
#10 is max number of interactions
#0.03 small epsolon?
#lk_params = dict(winSize=(1000,1000), maxLevel=1, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
lk_params = dict(winSize=(400,400), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [ ]:
#goal: track 10 points, flattening, and draw one line from prev to next points
cap = cv2.VideoCapture(0)
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
# points to track
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)
mask = np.zeros_like(prev_frame)
while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    if nextPts is not None and len(nextPts) > 0:
        good_new = nextPts[status==1]
        good_prev = prevPts[status==1]
    
        #zip() combines two arrays into tuple paring   
        for i, (new, prev) in enumerate(zip(good_new, good_prev)):
            x_new, y_new = new.ravel()
            x_prev, y_prev = prev.ravel()
        
            # Check if points are within image boundaries
            if 0 <= x_new < frame.shape[1] and 0 <= y_new < frame.shape[0] and 0 <= x_prev < frame.shape[1] and 0 <= y_prev < frame.shape[0]:
                mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_prev), int(y_prev)), (0,255,0), 3)
                frame = cv2.circle(frame, (int(x_new), int(y_new)), 8, (0,0,255), -1)

    
        img = cv2.add(frame, mask)
        cv2.imshow('tracking',img)
    
        k = cv2.waitKey(30) & 0xFF
        if k == 27:
            break
    
        prev_gray, frame_gray.copy()
        prevPts = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
cap.release()

[ WARN:0@0.444] global /croot/opencv-suite_1691620365762/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
